## Load Data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)
df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #Turning response into 0 and 1

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

## Fit and compare DP-EBM vs. standard EBM

In [ ]:
from interpret.privacy import DPExplainableBoostingClassifier, ExplainableBoostingClassifier
import time
from sklearn.metrics import roc_auc_score, accuracy_score

start = time.time()
dpebm = DPExplainableBoostingClassifier(epsilon=1, delta=1e-6)
_ = dpebm.fit(X_train, y_train)

dp_auroc = roc_auc_score(y_test, dpebm.predict_proba(X_test)[:, 1])
end = time.time()

print(f"DP EBM with eps: {dpebm.epsilon} and delta: {dpebm.delta} trained in {end - start:.2f} seconds with a test AUC of {dp_auroc:.3f}")


start = time.time()
ebm = ExplainableBoostingClassifier()
_ = ebm.fit(X_train, y_train)

ebm_auroc = roc_auc_score(y_test, ebm.predict_proba(X_test)[:, 1])
end = time.time()
print(f"EBM trained in {end - start:.2f} seconds with a test AUC of {ebm_auroc:.3f}")

## See differences in learned shape functions

In [ ]:
from interpret import show

show(ebm.explain_global(name='Standard EBM'))
show(dpebm.explain_global(name='DP EBM'))